Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Abstractive Summarization using BertSumAbs on CNN/DailyMails Dataset

## Summary

This notebook demonstrates how to fine tune BERT for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### BertSumAbs

BertSumAbs refers to an BERT-based abstractive summarization algorithm  in [Text Summarization with Pretrained Encoders](https://arxiv.org/abs/1908.08345) with [published examples](https://github.com/nlpyang/PreSumm). It uses the pretrained BERT model as encoder and finetune both encoder and decoder on a specific labeled summarization dataset like [CNN/DM dataset](https://github.com/harvardnlp/sent-summary). 

The figure below shows the comparison of architecture of the original BERT model (left) and BERTSUM (right), which BertSumAbs is built upon. For BERTSUM, a input document is split into sentences, and [CLS] and [SEP] tokens are inserted before and after each sentence. This resulting sequence is followed by the summation of three kinds of embeddings for each token before feeding into the transformer layers. The positional embedding used in BertSumAbs enables input length of more than 512, which is the  maximum input length for BERT model. 

It should be noted that the architecture only shows the encoder part. For decoder, BertSumAbs also uses a transformer with multiple layers and random initialization. As pretrained weights are used in the encoder, there is a mismatch in encoder and decoder which may result in unstable finetuning. Therefore, in fine tuning, BertSumAbs uses seperate optimizers for encoder and decoder, each uses its own scheduling. In text generation, techniques like trigram blocking and beam search can be used to improve model accuracy.
<img src="https://nlpbp.blob.core.windows.net/images/BertForSummarization.PNG">


## Before you start
Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = False, the notebook takes about 5 hours to run on a VM with 4 16GB NVIDIA V100 GPUs. Finetuning costs around 1.5 hours and inferecing costs around 3.5 hour.  Better performance can be achieved by increasing the MAX_STEPS.

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [abstractive_summarization_bertsum_cnndm_distributed_train.py](./abstractive_summarization_bertsum_cnndm_distributed_train.py) shows an example of how to use DDP.

* **Mixed Precision Training**:
Please note that by default this notebook doesn't use mixed precision training. Faster speed and larger batch size can be achieved when you set FP16 to True. Refer to  https://nvidia.github.io/apex and https://github.com/nvidia/apex) for details to use mixed precision training. Check the GPU model on your machine to see if it allows mixed precision training. Please also note that mixed precision inferencing is also enabled in the prediciton utility function. When you use mixed precision training and/or inferencing, the model performance can be slightly worse than the full precision mode.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
QUICK_RUN = False

In [39]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import BertSumAbs, BertSumAbsProcessor

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python

import pandas as pd
import pprint
import scrapbook as sb

## Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples. The length of the news articles is 781 tokens on average and the summaries are of 3.75 sentences and 56 tokens on average.

The significant part of data preprocessing only involve splitting the input document into sentences.

In [31]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [32]:
train_dataset, test_dataset = CNNDMSummarizationDataset(
            top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
        )

In [33]:
len(train_dataset)

287227

In [34]:
len(test_dataset)

11490

## Model Finetuning

In [8]:
# notebook parameters
# the cache path
CACHE_PATH = TemporaryDirectory().name

# model parameters
MODEL_NAME = "bert-base-uncased"
MAX_POS = 768
MAX_SOURCE_SEQ_LENGTH = 640
MAX_TARGET_SEQ_LENGTH = 140

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md. 
FP16 = False
if FP16:
    FP16_OPT_LEVEL="O2"
    
# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 3


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Learning rate
LEARNING_RATE_BERT=5e-4/2.0
LEARNING_RATE_DEC=0.05/2.0


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=10
SAVE_EVERY=500

# total number of steps for training
MAX_STEPS=1e2
   
if not QUICK_RUN:
    MAX_STEPS=5e3

WARMUP_STEPS_BERT=2000
WARMUP_STEPS_DEC=1000   


In [9]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_tgt_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [10]:
BATCH_SIZE_PER_GPU*NUM_GPUS

12

In [11]:

summarizer.fit(
        train_dataset,
        num_gpus=NUM_GPUS,
        batch_size=BATCH_SIZE_PER_GPU*NUM_GPUS,
        max_steps=MAX_STEPS,
        learning_rate_bert=LEARNING_RATE_BERT,
        learning_rate_dec=LEARNING_RATE_DEC,
        warmup_steps_bert=WARMUP_STEPS_BERT,
        warmup_steps_dec=WARMUP_STEPS_DEC,
        save_every=SAVE_EVERY,
        report_every=REPORT_EVERY*5,
        fp16=FP16,
        # checkpoint="saved checkpoint path"
)


device is cuda


Iteration:   0%|          | 0/23936 [00:00<?, ?it/s]/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iteration:   0%|          | 50/23936 [01:07<6:57:14,  1.05s/it] 

timestamp: 12/03/2020 19:42:23, average loss: 10.372837, time duration: 67.029438,
                            number of examples in current reporting: 600, step 50
                            out of total 5000


Iteration:   0%|          | 100/23936 [02:00<6:51:39,  1.04s/it]

timestamp: 12/03/2020 19:43:16, average loss: 6.238095, time duration: 53.688107,
                            number of examples in current reporting: 600, step 100
                            out of total 5000


...

timestamp: 12/03/2020 21:10:49, average loss: 2.801602, time duration: 53.101142,
                            number of examples in current reporting: 600, step 4950
                            out of total 5000


Iteration:  21%|██        | 4999/23936 [1:30:25<5:29:50,  1.05s/it]

timestamp: 12/03/2020 21:11:42, average loss: 2.808340, time duration: 53.435037,
                            number of examples in current reporting: 600, step 5000
                            out of total 5000
saving through pytorch to ./abstemp/bert-base-uncased_step_5000.pt


Iteration:  21%|██        | 5000/23936 [1:30:28<8:28:23,  1.61s/it]


saving through pytorch to ./abstemp/fine_tuned/bertsumabs.pt


In [12]:
summarizer.save_model(MAX_STEPS, os.path.join(CACHE_PATH, "./bertsumabs.pt"))

saving through pytorch to ./abstemp/./bertsumabs.pt


## Model Evaluation

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](summarization_evaluation.ipynb) for setup.
For the settings in this notebook with QUICK_RUN=False, you should get ROUGE scores close to the following numbers: <br />
``
{'rouge-1': {'f': 0.34819639878321873,
             'p': 0.39977932634737307,
             'r': 0.34429079596863604},
 'rouge-2': {'f': 0.13919271352557894,
             'p': 0.16129965067780644,
             'r': 0.1372938054050938},
 'rouge-l': {'f': 0.2313282318854973,
             'p': 0.26664667422849747,
             'r': 0.22850294283399628}}
 ``
 
 Better performance can be achieved by increasing the MAX_STEPS.

In [19]:
# del summarizer
checkpoint = torch.load("./abstemp/./bertsumabs.pt")
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)
summarizer.model.load_checkpoint(checkpoint['model'])

In [16]:
# clear cache
import gc; gc.collect()
torch.cuda.empty_cache()

In [27]:
len(train_dataset)

287227

In [25]:
QUICK_RUN
# TOP_N = 32
if not QUICK_RUN:
    TOP_N = len(test_dataset)
print(TOP_N)

32


In [35]:
TEST_TOP_N = 32
if not QUICK_RUN:
    TEST_TOP_N = len(test_dataset)
    
shortened_dataset= test_dataset.shorten(top_n=TEST_TOP_N)
src = shortened_dataset.get_source()
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset.get_target()]
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=32*4, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)


Generating summary:   0%|          | 0/90 [00:00<?, ?it/s]

dataset length is 11490


Generating summary: 100%|██████████| 90/90 [3:26:35<00:00, 128.17s/it]  


In [37]:
src[0]

['marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .',
 'marseille prosecutor brice robin told cnn that " so far no videos were used in the crash investigation . "',
 'he added , " a person who has such a video needs to immediately give it to the investigators . "',
 "robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps .",
 'all 150 on board were killed .',
 'paris match and bild reported that the video was recovered from a phone at the wreckage site .',
 'the two publications described the supposed video , but did not post it on their websites .',
 'the publications said that they watched the video , which was found by a source close to the investigation . "',
 'on

In [21]:
generated_summaries[0]

'french prosecutor : " so far no videos were used in the crash of germanwings flight 95fk .   new : " we can hear cries of my god \' in several languages , "   french lawyer says he was not aware of any video footage from the crash , " the report says .'

In [38]:
reference_summaries[0]

' marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports .    journalists at bild and paris match are " very confident " the video clip is real , an editor says .    andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says .  '

In [ ]:
rouge_scores = compute_rouge_python(cand=generated_summaries, ref=reference_summaries)


In [42]:
pprint.pprint(rouge_scores)

{'rouge-1': {'f': 0.34819639878321873,
             'p': 0.39977932634737307,
             'r': 0.34429079596863604},
 'rouge-2': {'f': 0.13919271352557894,
             'p': 0.16129965067780644,
             'r': 0.1372938054050938},
 'rouge-l': {'f': 0.2313282318854973,
             'p': 0.26664667422849747,
             'r': 0.22850294283399628}}


In [43]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_PATH):
    shutil.rmtree(CACHE_PATH, ignore_errors=True)